In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from pymongo import MongoClient
from dotenv import load_dotenv

In [2]:
# Nama database dan collection
db_name = 'youtube_rewind_indonesia'
collection_label = 'labeling_2016'
collection_name = 'full_labeling_2016'

# Memuat value dari file .env
load_dotenv()

mongodb_url = os.getenv('URL_SANDY')
local_url = os.getenv('URL_LOCAL')

#### Mengambil data dari MongoDB ke dalam dataframe

In [3]:
# Membuat koneksi ke MongoDB
client = MongoClient(local_url)
db = client[db_name]  # Ganti dengan nama database Anda
labeling_2016_collection = db[collection_label]
full_labeling_2016_collection = db[collection_name]

# Mengambil data dari koleksi MongoDB
labeled_data = list(labeling_2016_collection.find())
unlabeled_data = list(full_labeling_2016_collection.find())

# Konversi data menjadi dataframe
df_labeled = pd.DataFrame(labeled_data)
df_unlabeled = pd.DataFrame(unlabeled_data)

# Memisahkan fitur dan label dari data yang berlabel
X_labeled = df_labeled['textOriginal']
y_labeled = df_labeled['label']

# Menggunakan TfidfVectorizer untuk mengubah teks menjadi fitur
vectorizer = TfidfVectorizer()
X_labeled_tfidf = vectorizer.fit_transform(X_labeled)

# Melatih model menggunakan Naive Bayes
model = MultinomialNB()
model.fit(X_labeled_tfidf, y_labeled)

# Memproses data yang tidak berlabel
X_unlabeled = df_unlabeled['textOriginal']
X_unlabeled_tfidf = vectorizer.transform(X_unlabeled)

# Memprediksi label untuk data yang tidak berlabel
y_unlabeled_pred = model.predict(X_unlabeled_tfidf)

# Menambahkan prediksi ke dalam dataframe data tidak berlabel
df_unlabeled['predicted_label'] = y_unlabeled_pred

# Mengupdate koleksi MongoDB dengan prediksi label
for index, row in df_unlabeled.iterrows():
    full_labeling_2016_collection.update_one(
        {"_id": row["_id"]},
        {"$set": {"predicted_label": row["predicted_label"]}}
    )

print("Prediksi label untuk data tidak berlabel telah selesai dan diupdate dalam koleksi MongoDB.")

Prediksi label untuk data tidak berlabel telah selesai dan diupdate dalam koleksi MongoDB.


In [4]:
df_positif = df_unlabeled[df_unlabeled['predicted_label'] == 'positif']
df_negatif = df_unlabeled[df_unlabeled['predicted_label'] == 'negatif']
df_netral = df_unlabeled[df_unlabeled['predicted_label'] == 'netral']

In [5]:
df_positif

,_id,textOriginal,predicted_label
0,665189f294a2f3e28c752b16,eka gustiwana pengaruh besar ytrewind musik ny...,positif
1,665189f294a2f3e28c752b17,this felt so much beter to watch compared to y...,positif
3,665189f294a2f3e28c752b19,semua indah si acau datang,positif
7,665189f294a2f3e28c752b1d,anjir keren merinding cikay cantik pis,positif
8,665189f294a2f3e28c752b1e,best scene pap masuk manequin nya,positif
...,...,...,...
9978,665189f294a2f3e28c755210,tetap baik tetep salut buat ericko lim dyland ...,positif
9981,665189f294a2f3e28c755213,mis you rap lo bakal tetep jadi legend of youtube,positif
9982,665189f294a2f3e28c755214,habis nonton ytr langsung liat buat ngebanding...,positif
9995,665189f294a2f3e28c755221,may god job keren,positif


In [6]:
df_negatif

,_id,textOriginal,predicted_label
12,665189f294a2f3e28c752b22,tak tanding,negatif
24,665189f294a2f3e28c752b2e,kok ga young lex ama awkarin,negatif
32,665189f294a2f3e28c752b36,sat para konten creator kenal karya bukan drama,negatif
128,665189f294a2f3e28c752b96,bayang sih loe kalau youtube sama instagram ha...,negatif
159,665189f294a2f3e28c752bb5,rewind belah terlalu cringe,negatif
...,...,...,...
9753,665189f294a2f3e28c75512f,drama drama serang wkwk,negatif
9761,665189f294a2f3e28c755137,yutuber sekarang isi sampah,negatif
9763,665189f294a2f3e28c755139,kok jelek,negatif
9908,665189f294a2f3e28c7551ca,corona tak jarak dekat jaga jarak cuci tangan ...,negatif


In [7]:
df_netral

,_id,textOriginal,predicted_label
2,665189f294a2f3e28c752b18,stil man teman,netral
4,665189f294a2f3e28c752b1a,menit tengah rachel venya bukan,netral
5,665189f294a2f3e28c752b1b,tidak rewind,netral
6,665189f294a2f3e28c752b1c,kesini karna liat podcast om dedy sama chandra...,netral
10,665189f294a2f3e28c752b20,apa cuma sini nonton ulang cuma pgn liat scene...,netral
...,...,...,...
9993,665189f294a2f3e28c75521f,perhati akhir miawaug,netral
9994,665189f294a2f3e28c755220,gila yt rewind paling kenal video udah tonton ...,netral
9997,665189f294a2f3e28c755223,edhozel ada,netral
9998,665189f294a2f3e28c755224,ken ikut tahun,netral


#### Accuracy Test (soon)

In [8]:
# Jika data tidak berlabel memiliki label asli untuk keperluan evaluasi
if 'label' in df_unlabeled.columns:
    print("\nEvaluasi kinerja model:")
    print("Accuracy Score:", accuracy_score(df_unlabeled['label'], df_unlabeled['predicted_label']))
    print("Classification Report:\n", classification_report(df_unlabeled['label'], df_unlabeled['predicted_label']))
else:
    print("Data tidak berlabel tidak memiliki label asli untuk evaluasi.")

Data tidak berlabel tidak memiliki label asli untuk evaluasi.
